In [ ]:
from cosapp.drivers import NonLinearSolver, RungeKutta, MonteCarlo, LinearDoE
from cosapp.recorders import DataFrameRecorder
from cosapp.utils.distributions import Normal

from cosapp_lab.widgets import SysExplorer

from cpu.systems import CPUSystem

In [ ]:
cpu = CPUSystem("cpu")

In [ ]:
design = cpu.add_driver(NonLinearSolver('solver'))
design.extend(cpu.design_methods["exchanger_surface"])
design.runner.set_values({"fan.T_air": 40., "T_cpu": 80., "cpu.usage": 100.})

doe = cpu.add_driver(LinearDoE('doe'))

time_driver = doe.add_child(RungeKutta(order=3, history = True))
solver = time_driver.add_child(NonLinearSolver('solver', max_iter=10, factor=1.0))
time_driver.time_interval = (0, 30)
time_driver.dt = 0.5
time_driver.set_scenario(
    init={'T_cpu': 10.},
    values={
        "cpu.usage": "100 if time <20 else 0."
    }
)

doe.add_recorder(DataFrameRecorder(includes=['fan.tension', 'cpu.usage', 'T_cpu', 'fan.T_air']))
doe.add_input_var({'fan.T_air': {"lower": 30.0, "upper": 60.0, "count": 21}})

In [ ]:
cpu.run_drivers()
doe.recorder.data